In [ ]:
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data directory on Tale's drive
data_dir = '/content/drive/MyDrive/6.819_data'

In [ ]:
# Data directory on Sandra's drive
data_dir = '/content/drive/MyDrive/CV_Final_Proj/code/data/unzip_data'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
!pip install tqdm
from tqdm.notebook import tqdm
import os
import copy
import pandas as pd
import PIL 
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import skimage
  
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    print("You may want to try to use the GPU in Google Colab by clicking in:")
    print("Runtime > Change Runtime type > Hardware accelerator > GPU.")

Using the GPU!


In [ ]:
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    # random brightness and random contrast
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [ ]:
## Another way to process the dataset based on the txt split file
## Consistent with the original paper

batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data


class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample

In [ ]:
trainset = CovidCTDataset(root_dir=data_dir,
                          txt_COVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/COVID/trainCT_COVID.txt',
                          txt_NonCOVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                          transform= train_transformer)
valset = CovidCTDataset(root_dir=data_dir,
                          txt_COVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/COVID/valCT_COVID.txt',
                          txt_NonCOVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/NonCOVID/valCT_NonCOVID.txt',
                          transform= val_transformer)
testset = CovidCTDataset(root_dir=data_dir,
                          txt_COVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/COVID/testCT_COVID.txt',
                          txt_NonCOVID='/content/drive/MyDrive/CV_Final_Proj/code/data/Data-split/NonCOVID/testCT_NonCOVID.txt',
                          transform= val_transformer)
print(trainset.__len__())
print(valset.__len__())
print(testset.__len__())

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=False)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)

425
118
203


#### Modeling

In [ ]:
alpha = None
## alpha is None if mixup is not used
alpha_name = f'{alpha}'
device = 'cuda'

def make_optimizer(model, learning_rate, print_parameters=False):
    # Get all the parameters
    params_to_update = model.parameters()
    if print_parameters:
      print("Params to learn:")
      for name, param in model.named_parameters():
          if param.requires_grad == True:
              print("\t",name)

 
    optimizer = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)
    return optimizer

In [ ]:
### train function modified from problem sets
def train_model(model, optimizer, epoch, save_dir = "", save_all_epochs=False):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for batch_index, batch_samples in enumerate(data_loader):
        
                # move data to device
                inputs, labels = batch_samples['img'].to(device), batch_samples['label'].to(device) 

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    criterion = nn.CrossEntropyLoss()
                    loss = criterion(outputs, labels)

                    # torch.max outputs the maximum value, and its index
                    # Since the input is batched, we take the max along axis 1
                    # (the meaningful outputs)
                    _, preds = torch.max(outputs, 1)

                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)
            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_acc_history.append(epoch_acc)
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if save_all_epochs:
                torch.save(model.state_dict(), os.path.join(save_dir, f'weights_{epoch}.pt'))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # save and load best model weights
    torch.save(best_model_wts, os.path.join(save_dir, 'weights_best_val_acc.pt'))
    torch.save(model.state_dict(), os.path.join(save_dir, 'weights_last.pt'.format(epoch)))
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

In [ ]:
def initialize_model(model_name, num_classes, resume_from = None, use_pretrained = False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    input_size = 0
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "resnet50":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        # if use_pretrained:
        #   print('pretrained model')
        #   for param in model_ft.features.parameters():
        #     param.requires_grad = False
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        raise Exception("Invalid model name!")
    
    if resume_from is not None:
        print("Loading weights from %s" % resume_from)
        model_ft.load_state_dict(torch.load(resume_from))
    
    return model_ft, input_size

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!
model_name = 'resnet50'

# Number of classes in the dataset, normal, benign, malignant
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 
num_epochs = 20

# Learning rate
learning_rate = 0.002

### IO
# Path to a model file to use to start weights at
resume_from = None

# Whether to use a pretrained model, trained for classification in Imagenet-1k 
pretrained = True

# Save all epochs so that you can select the model from a particular epoch
save_all_epochs = False

# Whether to use early stopping (load the model with best accuracy), or not
early_stopping = True

# Directory to save weights to
# save_dir = models_dir + '/trained_model_1'
# os.makedirs(save_dir, exist_ok=True)

In [ ]:
def evaluate(model, data_loader, is_labelled = False, generate_labels = True, k = 5):
    # If is_labelled, we want to compute loss, top-1 accuracy and top-5 accuracy
    # If generate_labels, we want to output the actual labels
    # Set the model to evaluate mode
    model.eval()
    running_loss = 0
    running_top1_correct = 0
    running_top5_correct = 0
    predicted_labels = []
    gt_labels = []

    # Iterate over data.
    # TQDM has nice progress bars
    for batch_index, batch_samples in enumerate(data_loader):
        inputs, labels = batch_samples['img'].to(device), batch_samples['label'].to(device) 
        tiled_labels = torch.stack([labels.data for i in range(k)], dim=1) 
        # Makes this to calculate "top 5 prediction is correct"
        # [[label1 label1 label1 label1 label1], [label2 label2 label2 label label2]]

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            outputs = model(inputs)
            criterion = nn.CrossEntropyLoss()
            if is_labelled:
                loss = criterion(outputs, labels)

            # torch.topk outputs the maximum values, and their indices
            # Since the input is batched, we take the max along axis 1
            # (the meaningful outputs)
            _, preds = torch.topk(outputs, k=k, dim=1)
            if generate_labels:
                # We want to store these results
                nparr = preds.cpu().detach().numpy()
                predicted_labels.extend([list(nparr[i]) for i in range(len(nparr))])
                gt_labels.extend(np.array(labels.cpu()))

        if is_labelled:
            # statistics
            running_loss += loss.item() * inputs.size(0)
            # Check only the first prediction
            running_top1_correct += torch.sum(preds[:, 0] == labels.data)
            # Check all 5 predictions
            running_top5_correct += torch.sum(preds == tiled_labels)
        else:
            pass

    # Only compute loss & accuracy if we have the labels
    if is_labelled:
        epoch_loss = float(running_loss / len(data_loader.dataset))
        epoch_top1_acc = float(running_top1_correct.double() / len(data_loader.dataset))
        epoch_top5_acc = float(running_top5_correct.double() / len(data_loader.dataset))
    else:
        epoch_loss = None
        epoch_top1_acc = None
        epoch_top5_acc = None
    
    # Return everything
    return epoch_loss, epoch_top1_acc, gt_labels, predicted_labels  

In [ ]:
def get_eval_results(model, data_loader):
    model.eval()
    true_label_list = []
    outputs_list = []
    predicted_label_list = []
    original_image_list = []

    # TQDM has nice progress bars
    for batch_index, batch_samples in enumerate(data_loader):
        inputs, labels = batch_samples['img'].to(device), batch_samples['label'].to(device) 
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            outputs = model(inputs)
            true_label_list.append(labels)
            original_image_list.append(inputs)
            outputs_list.append(outputs)
            _, preds = torch.topk(outputs, k=1, dim=1)
            predicted_label_list.append(preds)
    return torch.concat(true_label_list).unsqueeze(-1).cpu().numpy(), \
           torch.concat(predicted_label_list).cpu().numpy(), \
           torch.softmax(torch.concat(outputs_list), dim=1).cpu().numpy(), \
           torch.concat(original_image_list).cpu().numpy()

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score

##### Resnet50

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'resnet50', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))

In [ ]:
def plot_confusion_matrix(y_label, y_pred, title='Confusion matrix'):
    cf_matrix = np.zeros((2, 2))    
    for actual_value, predicted_value in zip(y_label, y_pred):
        cf_matrix[actual_value, predicted_value] += 1
    cf_matrix_per = cf_matrix / np.sum(cf_matrix, axis=1, keepdims=True)
    
    group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix_per.flatten()]
    labels = [f"{v2}\n{v1}" for v1, v2 in
              zip(group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(2, 2)

    fig, ax = plt.subplots()  
    im = ax.imshow(cf_matrix_per, cmap="Blues")
    for i in range(2):
        for j in range(2):
            text = ax.text(j, i, labels[i][j], ha="center", va="center", 
                            color='w' if cf_matrix_per[i, j] > 0.5 else 'black')
    
    ax.set_title(title)
    ax.set_xticks(np.arange(2))
    ax.set_yticks(np.arange(2))
    ax.xaxis.set_ticklabels(['Non-COVID', 'COVID'])
    ax.yaxis.set_ticklabels(['Non-COVID', 'COVID'])
    ax.figure.colorbar(im, ax=ax)

    plt.show()

  
plot_confusion_matrix(y_label, y_pred, 'Confusion Matrix - ResNet50')

##### Resnet18

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'resnet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = 'resnet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))

##### densenet

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'densenet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = 'densenet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))

In [ ]:
y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
plot_confusion_matrix(y_label, y_pred, 'Confusion Matrix - DenseNet121')
print("f1 score is :", f1_score(y_label, y_pred))
print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

##### vgg

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'vgg', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = 'vgg', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))

In [ ]:
y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
plot_confusion_matrix(y_label, y_pred, 'Confusion Matrix - vgg')
print("f1 score is :", f1_score(y_label, y_pred))
print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

##### AlexNet

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'alexnet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = 'alexnet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))

In [ ]:
y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
plot_confusion_matrix(y_label, y_pred, 'Confusion Matrix - AlexNet')
print("f1 score is :", f1_score(y_label, y_pred))
print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

##### squeezenet

In [ ]:
test_acc, test_f1, test_auc = [], [], []

for i in range(5):
    model_1, input_size = initialize_model(model_name = 'squeezenet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
    model_1 = model_1.to(device)

    optimizer_1 = make_optimizer(model_1, learning_rate)
    trained_model_1, validation_history_1, train_history_1 = train_model(model_1, optimizer_1, num_epochs)

    # Load your final model, that we will use for the rest of the PSET.
    if early_stopping:
        weights_file = '' + 'weights_best_val_acc.pt'
    else:
        weights_file = '' + 'weights_last.pt'

    model_1, _ = initialize_model(model_name = 'squeezenet', num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

    # Move the model to the gpu if needed
    model_1 = model_1.to(device)

    # Load weights for model_yours
    model_1.load_state_dict(torch.load(weights_file))

    # Get data on the validation set

    val_loss_yours, val_top1_yours, _, val_labels_yours = evaluate(model_1, val_loader, is_labelled = True, generate_labels = True, k = 1)
    # Get predictions for the test set
    test_loss_yours, test_top1_yours, _, test_labels_yours = evaluate(model_1, test_loader, is_labelled = True, generate_labels = True, k = 1)

    test_acc.append(test_top1_yours)

    print("Our Trained model: ")
    print("Val Top-1 Accuracy: {}".format(val_top1_yours))
    print("Test Top-1 Accuracy: {}".format(test_top1_yours))

    y_label, y_pred, outputs, inputs =  get_eval_results(model_1, test_loader)
    test_f1.append(f1_score(y_label, y_pred))
    test_auc.append(roc_auc_score(y_label, outputs[:, 1]))

    print("f1 score is :", f1_score(y_label, y_pred))
    print("AUC score is ", roc_auc_score(y_label, outputs[:, 1]))

In [ ]:
print(np.mean(test_acc), np.std(test_acc))
print(np.mean(test_f1), np.std(test_f1))
print(np.mean(test_auc), np.std(test_auc))